In [1]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import logging
import torch
import torch.nn.functional as F
from torch import nn
from sklearn.cluster import KMeans
from lora import LoRA_sam 
import copy

# Function to visualize masks
def save_anns(image, anns, save_path):
    plt.figure(figsize=(20, 20))
    plt.imshow(image)
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:, :, 3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)
    plt.axis('off')
    plt.savefig(save_path)
    plt.close()

# Save the mask to the specified folder
def write_masks_to_folder(image, masks, path, image_suffix):
    save_anns(image, masks, os.path.join(path, 'full_mask' + image_suffix))
    for i, mask_data in enumerate(masks):
        mask = mask_data["segmentation"]
        filename = f"{i}" + image_suffix
        cv2.imwrite(os.path.join(path, filename), mask * 255)
    return

# Compute the bounding box of the mask
def calculate_bbox(mask):
    y_indices, x_indices = np.where(mask > 0)
    if len(y_indices) == 0 or len(x_indices) == 0:
        return None  # Empty mask
    x_min, x_max = x_indices.min(), x_indices.max()
    y_min, y_max = y_indices.min(), y_indices.max()
    return [x_min, y_min, x_max, y_max]

# Compute the IoU (Intersection over Union) of two bounding boxes
def calculate_iou(bbox1, bbox2):
    x1_min, y1_min, x1_max, y1_max = bbox1
    x2_min, y2_min, x2_max, y2_max = bbox2

    inter_xmin = max(x1_min, x2_min)
    inter_ymin = max(y1_min, y2_min)
    inter_xmax = min(x1_max, x2_max)
    inter_ymax = min(y1_max, y2_max)

    inter_width = max(0, inter_xmax - inter_xmin + 1)
    inter_height = max(0, inter_ymax - inter_ymin + 1)
    inter_area = inter_width * inter_height

    bbox1_area = (x1_max - x1_min + 1) * (y1_max - y1_min + 1)
    bbox2_area = (x2_max - x2_min + 1) * (y2_max - y2_min + 1)

    iou = inter_area / float(bbox1_area + bbox2_area - inter_area)
    return iou

# NMS (Non-Maximum Suppression) filtering function
def nms_filtering_optimized(cluster_indices, features, mask_dir, file_names, kmeans, labels, iou_threshold=0.3):
    bboxes = []
    distances = []
    
    for idx in cluster_indices:
        mask_name = os.path.splitext(file_names[idx])[0] + '.png'
        mask_path = os.path.join(mask_dir, mask_name)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        bbox = calculate_bbox(mask)
        if bbox is not None:
            bboxes.append((bbox, idx))
            distance = np.linalg.norm(features[idx] - kmeans.cluster_centers_[labels[idx]])
            distances.append(distance)
    
    sorted_indices = sorted(range(len(distances)), key=lambda i: distances[i])
    keep_indices = []
    while sorted_indices:
        current_idx = sorted_indices.pop(0)
        keep_indices.append(bboxes[current_idx][1])
        current_bbox = bboxes[current_idx][0]
        
        sorted_indices = [
            i for i in sorted_indices
            if calculate_iou(current_bbox, bboxes[i][0]) <= iou_threshold
        ]
    
    return keep_indices

# 设置模型参数和路径
sam_checkpoint = '/home/a6/vis/LDS/SAM/weights/sam_vit_h_4b8939.pth'
lora_sam_checkpoint = '/home/a6/vis/LDS/SAM/logs/best_lora_cocenter_rank32.safetensors' 
model_type = 'vit_h'
device = 'cuda'
image_folder = '/home/a6/vis/LDS/segmentation/segformer-pytorch-master/VOCdevkit_pre/VOC2007/JPEGImages'
val_file = 'train.txt'
predict_dir = 'predict'

# 创建模型，加载SAM权重用于生成掩码
sam_model = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam_model.to(device=device)
original_sam_model = copy.deepcopy(sam_model)

# 使用LoRA微调的SAM模型生成图像的嵌入特征
lora_sam_model = LoRA_sam(sam_model, rank=32)

# 加载 LoRA 的权重
lora_sam_model.load_lora_parameters(lora_sam_checkpoint)
lora_sam_model.to(device)

# 使用原始模型生成掩码
mask_generator = SamAutomaticMaskGenerator(
    model=original_sam_model,
    points_per_side=64,
    pred_iou_thresh=0.86,
    stability_score_thresh=0.92,
    crop_n_layers=2,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,
)

# 读取val.txt文件中的图像名称
with open(val_file, 'r') as file:
    image_names = [line.strip() for line in file.readlines()]

# 遍历图像名称列表
for image_name in image_names:
    image_path = os.path.join(image_folder, image_name + '.jpg')
    
    if not os.path.exists(image_path):
        print(f"Image {image_path} not found, skipping.")
        continue

    # 加载图像
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (1024, 1024))

    # 生成掩码
    masks = mask_generator.generate(image)
    print(f'{image_name}: Number of masks generated: {len(masks)}')

    # 指定保存掩码的目录
    masks_dir = f'{predict_dir}/masks/{image_name}'
    os.makedirs(masks_dir, exist_ok=True)

    # 日志设置
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    output_dir = f'{predict_dir}/outputs'
    mask_vec_npy_dir = os.path.join(output_dir, 'npy_masks', image_name)
    os.makedirs(mask_vec_npy_dir, exist_ok=True)

    # 将掩码保存到指定目录
    write_masks_to_folder(image, masks, masks_dir, '.png')

    # 转换图像格式以适应模型输入
    image_torch = torch.as_tensor(image, device=device)
    transformed_image = image_torch.permute(2, 0, 1).contiguous()[None, :, :, :]
    input_image = lora_sam_model.sam.preprocess(transformed_image)
    with torch.no_grad():
        image_embedding = lora_sam_model.sam.image_encoder(input_image)  # [B, C, H, W]

    b, c, h, w = image_embedding.shape

    # 处理每个掩码并保存嵌入特征
    mask_files = [f for f in os.listdir(masks_dir) if f.endswith('.png') and f != 'full_mask.png']
    mask_vecs = []
    for mask_file in mask_files:
        mask_path = os.path.join(masks_dir, mask_file)
        mask_name = os.path.splitext(mask_file)[0]
        
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = mask.astype(np.float32) / 255.0
        mask = torch.as_tensor(mask, device=device)[None, None, :, :]
        
        # 缩放特征和掩码
        rescale_factor = 4
        t1, t2 = int(mask.shape[2] / rescale_factor), int(mask.shape[3] / rescale_factor)
        features_rescale = F.interpolate(image_embedding, size=[t1, t2], mode='bilinear')
        mask_rescale = F.interpolate(mask, size=[t1, t2], mode='bilinear')

        masked_feature = torch.mul(features_rescale, mask_rescale)
        masked_feature = masked_feature.view(b, c, -1)
        non_zero_count = torch.count_nonzero(masked_feature, dim=2)
        
        # 计算掩码的平均特征向量
        masked_avg_vec = masked_feature.sum(dim=2) / non_zero_count
        masked_avg_vec[torch.isnan(masked_avg_vec)] = 0
        
        # 保存掩码特征向量
        npy_data = masked_avg_vec.detach().cpu().numpy()
        single_mask_vec_path = os.path.join(mask_vec_npy_dir, mask_name + '.npy')
        np.save(single_mask_vec_path, npy_data)
        mask_vecs.append(npy_data)

    # 将特征向量转换为 NumPy 数组
    if len(mask_vecs) > 0:
        mask_vecs = np.array(mask_vecs).squeeze()

        # 对每张图像的掩码进行2分类聚类
        kmeans = KMeans(n_clusters=2, random_state=0, n_init=100)
        labels = kmeans.fit_predict(mask_vecs)

        # 执行NMS过滤
        cluster_indices_1 = np.where(labels == 0)[0]
        cluster_indices_2 = np.where(labels == 1)[0]

        filtered_cluster_indices_1 = nms_filtering_optimized(cluster_indices_1, mask_vecs, masks_dir, mask_files, kmeans, labels)
        filtered_cluster_indices_2 = nms_filtering_optimized(cluster_indices_2, mask_vecs, masks_dir, mask_files, kmeans, labels)

        # 创建图像对应的结果保存路径
        results_dir = os.path.join(f'{predict_dir}/results', image_name)
        os.makedirs(results_dir, exist_ok=True)

        # 合并并保存聚类后的掩码
        for cluster_id in range(2):  # 对每个聚类进行处理
            combined_mask = np.zeros((1024, 1024), dtype=np.float32)
            if cluster_id == 0:
                selected_indices = filtered_cluster_indices_1
            else:
                selected_indices = filtered_cluster_indices_2

            for idx in selected_indices:
                mask_path = os.path.join(masks_dir, mask_files[idx])
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) / 255.0
                combined_mask = np.maximum(combined_mask, mask)
            
            # 保存聚类后的掩码
            save_path = os.path.join(results_dir, f'{image_name}-{cluster_id + 1}.png')
            cv2.imwrite(save_path, combined_mask * 255)

        # 选择掩码数量较少的那张图像，并保存到一个新的文件夹
        if len(filtered_cluster_indices_1) <= len(filtered_cluster_indices_2):
            selected_mask = filtered_cluster_indices_1
        else:
            selected_mask = filtered_cluster_indices_2

        combined_mask = np.zeros((1024, 1024), dtype=np.float32)
        for idx in selected_mask:
            mask_path = os.path.join(masks_dir, mask_files[idx])
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) / 255.0
            combined_mask = np.maximum(combined_mask, mask)

        # 保存到新文件夹中，文件名为原图文件名
        final_dir = os.path.join(predict_dir, 'final_results')
        os.makedirs(final_dir, exist_ok=True)
        final_save_path = os.path.join(final_dir, f'{image_name}.png')
        cv2.imwrite(final_save_path, combined_mask * 255)

print("Processing complete.")


/home/a6/anaconda3/envs/LDS/lib/python3.9/site-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


X26-1498.40m1-: Number of masks generated: 762
X53-1697.98m2-: Number of masks generated: 936
X26-1604.43m1-: Number of masks generated: 564
X40-2001.10m1-: Number of masks generated: 510
X18-1369.20m1-: Number of masks generated: 1014
X159-1253.78m1-: Number of masks generated: 734
X51-1633.71m1-: Number of masks generated: 531
X41-1936.00m1-: Number of masks generated: 387
X12-2064.00m1-: Number of masks generated: 349
X53-1696.79m1-: Number of masks generated: 548
X21-1655.27m1-: Number of masks generated: 512
X45-1772.50m1-: Number of masks generated: 299
X48-1547.90m1-: Number of masks generated: 1158
X151-1845.31m1-: Number of masks generated: 515
X26-1930.40m1-: Number of masks generated: 550
X16-1870.11m1-: Number of masks generated: 365
X46-1565.86m2-: Number of masks generated: 656
X48-1623.65m1-: Number of masks generated: 737
X45-1874.50m1-: Number of masks generated: 354
X31-1581.50m1-: Number of masks generated: 531
X51-1859.10m1-: Number of masks generated: 1613
X53-1480